# Escuela Politécnica Nacional

### Objetivos

Este proyecto tiene como objetivo medir el tiempo promedio que los estudiantes de la EPN que ingresarone el semestre 2012-A (pertenecientes a la primera promoción del SENECYT)se tardaron en culminar su carrera y encontrar si existe una relación entre la carrera y el tiempo de graduación.
Se debe tener en cuenta que como se trabaja con la lista de planes de tesis, los estudiantes considerados en el anális son solo los que se graduan mediante el proceso de titulación por proyecto.

Librerías utilizadas

In [1]:
import tabula
import pandas as pd
from datetime import datetime
import math

### Parámetros y configuraciones

In [2]:
FILE_PATH_STUDENT = "inputs/APROBADOS INGENIERIA_2012.pdf"
FILE_PATH_STUDENT_HTML = "inputs/planesTesis.html"
FILE_PATH_STUDENTS_CSV = "epnStudents2021.csv"

## Extracción

Para realizar este análisis, se toma información de dos fuentes: 
- Lista de estudiantes aprobados para inicar el prepólitecnico en el semestre 2012 - A en formato PDF 
-  Lista de todos los planes aprovados disponible en la página del saew [saew](https://saew.epn.edu.ec/SAETESIS/BusquedaPlanTesis.aspx) en formato HTML

### Extración de información de estudiantes aprovados para el 2012-A en formato PDF

Leer información del pdf y convertir en archivo CSV

In [3]:
def getInfoFromPdf():
    tabula.convert_into(FILE_PATH_STUDENT, 'epnStudents2021.csv', pages='all')

Método para generar el dataset a partir un CSV

In [4]:
def getDatasetFromCSVFile(file_path_param):
    data_set = pd.read_csv(FILE_PATH_STUDENTS_CSV)
    return data_set

Método para generar el correo electrónico de los estudiantes, este campo será utilizado para hacer match
con el otro dataset

In [5]:
def addEmail(df):
    names = df['NOMBRES']
    apellidos = df['APELLIDOS']

    # Generar los correos electrónicos
    correos = []
    for nombre, apellido in zip(names, apellidos):
        nombre_partes = nombre.lower().split()
        apellido_partes = apellido.lower().split()

        # Construir el correo electrónico
        correo = f"{nombre_partes[0]}.{apellido_partes[0]}@epn.edu.ec"
        correos.append(correo)

    df['Email'] = correos
    return df

Método que retorna un data set con la información <br>
[Nombre , Carrera, Email]

In [6]:
def getInfo2012Students():
    getInfoFromPdf()
    info_students_from_pdf = getDatasetFromCSVFile(FILE_PATH_STUDENTS_CSV)
    info_studens_with_email = addEmail(info_students_from_pdf)
    return info_studens_with_email
    

In [7]:
students_2012_info = getInfo2012Students()

In [8]:
#print(students_2012_info)

### Extración de información los planes de proyecto de titulación en formato HTML

Método para sacar la info del html V2

Metodo para tomar solo las columnas necesaraias

In [9]:
def generar_dataframe(datos, columnas_necesarias):
    df = pd.DataFrame(datos, columns=columnas_necesarias)
    return df

In [10]:
def getEpnMails(email_str):
    all_emails = list(map(lambda email: email.replace(";",""), email_str.split()))
    epn_mails = list(filter(lambda email: "@epn.edu.ec" in email, all_emails))
    return ' '.join(epn_mails)

In [11]:
def getInfo(df):
    return generar_dataframe(df, ['Carrera', 'FecGradoOral', 'Email'])

Método que genera un dataFrame solo con los correos de la EPN

In [12]:
def getOnlyEpnMails(df):
    data_frame_all_emails = getInfo(df)
    email_col = data_frame_all_emails['Email']
    epn_email_col = []
    for emails in email_col:
        epn_mails = getEpnMails(emails)
        epn_email_col.append(epn_mails)
    
    data_frame_all_emails['EpnEmails'] = epn_email_col
    return data_frame_all_emails

Todo: Lo que hay que hacer es: 
1 . Generar un nuevo dataFrame
2. Las filas que ingresan directamente son las que tienen un solo correo
3. Las filas que tiene mas de un correo se debe duplicar, y cada nueva fila debe tener solo un correo de los anteriores

In [13]:
dataframes = pd.read_html(FILE_PATH_STUDENT_HTML)
students_and_tesis = getOnlyEpnMails(dataframes[0])



Método que valida si hay más de dos correos en la columa 'EpnEmails', replica la información y deja un solo correo

In [53]:
def getUniqueEmail(df):
    df_destino = pd.DataFrame()
    for index, row in df.iterrows():
        epn_emails = row['EpnEmails'].split(' ')
        epn_emails = [email.strip() for email in epn_emails]
        if len(epn_emails) == 1:
            fila = df.iloc[index]
            fila['Epn_mail_unique'] = str(epn_emails[0])
            df_destino = pd.concat([df_destino, fila.to_frame().T])
        else:
            for idx, epn_mail in enumerate(epn_emails):
                fila = df.iloc[index]
                fila['Epn_mail_unique'] = str(epn_mail)
                df_destino = pd.concat([df_destino, fila.to_frame().T])

    return df_destino
    

In [55]:
df3 = getUniqueEmail(students_and_tesis)

In [56]:
print(df3)

                                                Carrera        FecGradoOral  \
0                    (RRA) ELECTRONICA Y AUTOMATIZACION                 NaN   
0                    (RRA) ELECTRONICA Y AUTOMATIZACION                 NaN   
1     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
2     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
3     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
...                                                 ...                 ...   
6596             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL  17/08/2018 0:00:00   
6597             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL                 NaN   
6598             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL                 NaN   
6599      TECNOLOGIA EN PROCESOS DE PRODUCCION MECANICA  15/06/2017 0:00:00   
6600      TECNOLOGIA EN PROCESOS DE PRODUCCION MECANICA  15/06/2017 0:00:00   

                                                  E

-------

In [57]:
# print(students_and_tesis)

## Transformación

Para el proceso de transformación se procede a intersecar los dos dataframes, mediante el correo institucional

In [68]:
def combinar_dataframes(df1, df2):
    nuevo_df = pd.merge(df1, df2, left_on='Email', right_on='Epn_mail_unique')
    return nuevo_df

Método que genera un nuevo dataFrame realizando una intersección entre los dos dataFrames en base al campo email

In [69]:
def generar_dataframe_interseccion(df1, df2):
    # Obtiene los correos electrónicos únicos en ambos DataFrames
    emails_df1 = set(df1['Email'])
    emails_df2 = set(df2['Epn_mail_unique'])
    
    
    # Encuentra los correos electrónicos que se repiten en ambos DataFrames
    emails_repetidos = list(emails_df1.intersection(emails_df2))
    
    # Filtra los DataFrames originales utilizando los correos electrónicos repetidos
    df1_interseccion = df1[df1['Email'].isin(emails_repetidos)]
    df2_interseccion = df2[df2['Epn_mail_unique'].isin(emails_repetidos)]
    
    # Crea un nuevo DataFrame con los valores que se repiten en ambos DataFrames
    dataframe_interseccion = pd.concat([df1_interseccion, df2_interseccion], ignore_index=True)
    
    return dataframe_interseccion

In [72]:
merge_info = combinar_dataframes(students_2012_info,df3)

In [73]:
print(merge_info.iloc[0])

APELLIDOS                                              ABARCA ROBLES
NOMBRES                                                  JUAN DANIEL
CARRERA                                    INGENIERIA AGROINDUSTRIAL
Email_x                                       juan.abarca@epn.edu.ec
Carrera                                    INGENIERIA AGROINDUSTRIAL
FecGradoOral                                      30/04/2020 0:00:00
Email_y            juan.abarca.r.1995@hotmail.com; juan.abarca@ep...
EpnEmails                                     juan.abarca@epn.edu.ec
Epn_mail_unique                               juan.abarca@epn.edu.ec
Name: 0, dtype: object


Método que transforma la fecha

In [74]:
def convertir_fecha(cadena_fecha):
    formato = "%d/%m/%Y %H:%M:%S"
    fecha = datetime.strptime(cadena_fecha, formato)
    return fecha

Calular el tiempo

In [130]:
def restar_fechas(fecha1, fecha2):
    formato = '%Y-%m-%d %H:%M:%S'
    
    # Convertir las cadenas a objetos datetime
    fecha1_dt = datetime.strptime(fecha1, formato)
    fecha2_dt = datetime.strptime(fecha2, formato)
    
    # Calcular la diferencia de tiempo
    diferencia = fecha2_dt - fecha1_dt
    
    # Calcular la diferencia en años
    diferencia_anios = diferencia.days / 365.25
    
    # Retornar la diferencia en años
    return diferencia_anios


Método que toma le fecha en la que se registró la defensa de grado oral 'FecGradoOral' y se procede a calcular el tiempo transcurrido desde 2012


In [131]:
def calcular_anios(fecha1, fecha2):
    anios = fecha1.year - fecha2.year
    if fecha1.month < fecha2.month or (fecha1.month == fecha2.month and fecha1.day < fecha2.day):
        anios -= 1
    return anios

In [136]:
def add_elapsed_time_column(df):
    # Constantes
    formato = "%d/%m/%Y %H:%M:%S"
    date_time_2012 = datetime(2012, 1, 1, 0, 0, 0)
    
    oral_defense_date_string = df['FecGradoOral']
    elapsed_date_from_2012 = []
    
    for line in oral_defense_date_string:
        if isinstance(line, str):
            fecha_datetime = datetime.strptime(line, formato)
            elapsed_time = calcular_anios(fecha_datetime, date_time_2012)
            elapsed_date_from_2012.append(elapsed_time)
        else:
            elapsed_date_from_2012.append(0)
    
    df['TiempoGraduación'] = elapsed_date_from_2012
    return df
            

In [137]:
print(merge_info)

Empty DataFrame
Columns: [APELLIDOS, NOMBRES, CARRERA, Email, Carrera, FecGradoOral, EpnEmails, TiempoGraduación]
Index: []


In [133]:
clean_info = add_elapsed_time_column(merge_info)

In [134]:
print(clean_info)

Empty DataFrame
Columns: [APELLIDOS, NOMBRES, CARRERA, Email, Carrera, FecGradoOral, EpnEmails, TiempoGraduación]
Index: []


## Load

Vamos a realizar un gráfico que nos indique que el timepo de la carrera

In [135]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
agrupado = clean_info.groupby(['CARRERA'])
print("***")
print(agrupado.get_group('INGENIERIA AGROINDUSTRIAL'))
print("---")
print(agrupado.get_group('INGENIERIA AGROINDUSTRIAL')['TiempoGraduación'].mean())
print("***")


***


KeyError: 'INGENIERIA AGROINDUSTRIAL'